In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import morphology
%matplotlib inline

In [2]:
from myfunc import clip_background,cv2plot,set_borders_zero,resize_by_ratio,morph_line

### load src image

In [8]:
# SMED30004894.jpg
# SMED30006949
# SMED30033119
# SMED30021202
# 参考了https://blog.csdn.net/weixin_44524040/article/details/90350701
# 不过我稍微改了一下绿色的比例。。

In [9]:
src = cv2.imread('SMED30021202.jpg')
cv2plot(src,'src')

### Extract Fish image

In [10]:
fsrc = np.array(src, dtype=np.float32) / 255.0
(b, g, r) = cv2.split(fsrc)
gray = 1 * g - b - r
# select values between 0 1
res = np.clip(gray, 0, 1)

# 数值大于0.x的都设定为1
rnd = np.ceil(res-0.5)
# cv2plot(rnd,'rnd')
np.unique(rnd)

array([-0.,  1.], dtype=float32)

###  now clip body shape 

In [11]:
b1 = np.clip(b, 0, 1)
b2 = np.ceil(b1-0.5)
# cv2plot(b2,'b2')
# np.unique(b2)

In [12]:
len(np.argwhere(b2[0]==1))

1

In [13]:
# set first&last 3 row&col as 0

b2 = set_borders_zero(b2,3)

In [14]:
kernel = np.ones((30, 30), np.uint8)   # or cv2.morphologyEx(b2, cv2.MORPH_CLOSE, kernel)

# 虚线膨胀链接
d_im = cv2.dilate(b2, kernel, iterations=1)   #b2 已经二值化

#Todo:取中线/骨架
d_im = morphology.skeletonize(d_im).astype(np.float32)

# 填充
mymask = np.zeros((d_im.shape[0]+2,d_im.shape[1]+2))
mymask = np.mat(mymask,np.uint8)
d_im = cv2.floodFill(d_im,mymask,(0,0),1)[1] #图像左上角开始，大背景填充1

#线条圆润
for i in range(5):
    d_im = cv2.medianBlur(d_im, 5)
    d_im = cv2.GaussianBlur(d_im,(5,5),0)

#二值化-if blurred
d_im = np.ceil(d_im-0.5)

# cv2plot(d_im,'body shape')

### clip lines&rows sums zero & check if is correct

In [15]:
# chk = rnd + d_im
# cv2plot(chk,'chk')

In [16]:
d_im,rnd = clip_background(d_im,rnd,denoise1 = 5,denoise2 = 3)

In [17]:
chk = rnd + d_im
cv2plot(chk,'chk body shape')

### Affine Transformation

In [18]:
# load target shape
tar = cv2.imread('myday10.png',0)
tar = cv2.pyrUp(tar, dstsize=(2 * tar.shape[1], 2 * tar.shape[0]))  # 好像只能2倍增加
tar = cv2.pyrUp(tar, dstsize=(2 * tar.shape[1], 2 * tar.shape[0]))  # 好像只能2倍增加
tar = tar/255
tar = cv2.threshold(tar,0.4,1,cv2.THRESH_BINARY)[1]
tar,_ = clip_background(tar,[],denoise1 = 5,denoise2 = 3)
len(tar[0])

252

In [19]:
# np.unique(tar)
# cv2plot(tar)

In [20]:
# resize d_im,rnd to target size
d_im = cv2.resize(d_im, (tar.shape[1],tar.shape[0]), interpolation = cv2.INTER_NEAREST)
rnd = cv2.resize(rnd, (tar.shape[1],tar.shape[0]), interpolation = cv2.INTER_NEAREST)
# cv2plot(rnd+d_im)

In [21]:
# extract skeleton -- marked as 1   TODO: 把涡虫摆正拉直
src_skl = morphology.skeletonize(d_im*-1+1).astype(np.float32)
tar_skl = morphology.skeletonize(tar*-1+1).astype(np.float32)

In [22]:
# resize it
new_d_im = np.ndarray(tar.shape)
new_rnd = np.ndarray(tar.shape)
for i in range(len(tar)):
    morph,submorph = morph_line(d_im[i],rnd[i],tar[i])
    new_d_im[i,:]=morph
    new_rnd[i,:]=submorph
    

In [23]:
cv2plot(new_rnd+new_d_im)